In [1]:
# allows to import own functions
import sys, os, re
root_project = re.findall(r'(^\S*TFM-master)', os.getcwd())[0]
sys.path.append(root_project)

from src.utils.help_func import results_searchcv, make_train_val_test, errors_distribution

In [2]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from scipy.stats import uniform, expon, randint, loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
import graphviz

In [3]:
df_i = pd.read_pickle(f'{root_project}/data/processed/features_model_rev15.pickle')
df_i

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,limit_deceased,n_closed,react_time,total_infected,total_deceased,total_recovered,betweenness,degree,closeness,country_pop
0,KWT,6.525694,0.599376,3.911348,0.798341,2.547143e+02,1.060500e+03,0.477566,0.802923,1.926471,534,4,29,5.028065e+09,3.808452e+09,1.219641e+09,0.001572,0.234234,0.484716,4137309.0
1,BRA,18.335226,0.276712,5.073580,0.032782,1.939390e+07,7.942701e+05,0.074366,0.068893,0.137342,22,19,17,7.511949e+09,1.071266e+09,6.440884e+09,0.010511,0.297297,0.502262,209469333.0
2,POL,11.298080,0.529086,5.977660,0.642468,7.377071e+03,3.060900e+04,0.340147,0.286925,1.847295,754,4,6,6.982598e+09,5.540089e+09,1.442559e+09,0.001542,0.360360,0.508009,37978548.0
3,MAC,14.685925,0.613497,9.009775,0.854669,3.617857e+02,1.704500e+03,0.524975,0.779994,4.632404,841,4,11,5.182405e+09,4.586775e+09,5.956527e+08,0.000000,0.081081,0.378194,631636.0
4,TUR,16.144391,0.174782,2.821745,0.649911,2.349075e+07,2.472220e+07,0.253842,0.105524,0.717261,321,13,28,7.503404e+09,4.865481e+09,2.653095e+09,0.043460,0.833333,0.606557,82319724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144045,LTU,19.059507,0.898042,17.116245,0.940615,2.285714e+00,4.000000e+00,0.843750,6.750000,13.500000,992,7,1,5.297120e+05,4.987320e+05,3.089800e+04,0.000126,0.234234,0.448485,2789533.0
144046,VCT,3.033011,0.684258,2.075362,0.256749,1.281286e+03,1.516875e+03,0.187423,0.277050,0.326186,309,15,7,4.422700e+09,1.537425e+09,2.885307e+09,0.000035,0.040541,0.312236,110210.0
144047,KWT,15.849174,0.464848,7.367461,0.471923,1.014680e+05,2.978810e+05,0.244441,0.194284,1.496761,793,15,27,7.502382e+09,5.826655e+09,1.675829e+09,0.001572,0.234234,0.484716,4137309.0
144048,TUV,16.451820,0.509182,8.376964,0.775358,7.477857e+02,5.400000e+01,0.821091,39.796296,6.188625,569,19,29,6.800794e+09,5.893454e+09,9.073601e+08,0.000000,0.009009,0.296395,11646.0


In [4]:
# df_i = df_i.sample(1000) # par entrenar con menos muestras


features = ['Tr', 'inf_pow_1', 'inf_pow_2', 'mort_pow_1', 'mort_pow_2', 'mort_pow_3', 'limit_deceased',
           'n_closed', 'react_time','total_deceased', 'betweenness', 'degree', 'closeness', 'country_pop']

features = ['Tr', 'inf_pow_1', 'inf_pow_2', 'mort_pow_3', 'limit_deceased',
           'n_closed', 'react_time','total_deceased', 'betweenness', 'degree', 'closeness', 'country_pop']

df = df_i[features]

size_data = int(len(df) / 1000)
print(size_data)
df

144


,Tr,inf_pow_1,inf_pow_2,mort_pow_3,limit_deceased,n_closed,react_time,total_deceased,betweenness,degree,closeness,country_pop
0,3.911348,2.547143e+02,1.060500e+03,1.926471,534,4,29,3.808452e+09,0.001572,0.234234,0.484716,4137309.0
1,5.073580,1.939390e+07,7.942701e+05,0.137342,22,19,17,1.071266e+09,0.010511,0.297297,0.502262,209469333.0
2,5.977660,7.377071e+03,3.060900e+04,1.847295,754,4,6,5.540089e+09,0.001542,0.360360,0.508009,37978548.0
3,9.009775,3.617857e+02,1.704500e+03,4.632404,841,4,11,4.586775e+09,0.000000,0.081081,0.378194,631636.0
4,2.821745,2.349075e+07,2.472220e+07,0.717261,321,13,28,4.865481e+09,0.043460,0.833333,0.606557,82319724.0
...,...,...,...,...,...,...,...,...,...,...,...,...
144045,17.116245,2.285714e+00,4.000000e+00,13.500000,992,7,1,4.987320e+05,0.000126,0.234234,0.448485,2789533.0
144046,2.075362,1.281286e+03,1.516875e+03,0.326186,309,15,7,1.537425e+09,0.000035,0.040541,0.312236,110210.0
144047,7.367461,1.014680e+05,2.978810e+05,1.496761,793,15,27,5.826655e+09,0.001572,0.234234,0.484716,4137309.0
144048,8.376964,7.477857e+02,5.400000e+01,6.188625,569,19,29,5.893454e+09,0.000000,0.009009,0.296395,11646.0


In [5]:
X_train_val, y_train_val, X_test, y_test = make_train_val_test(df, out_mode=1)

Train_validation set: (115240, 11)
Test set: (28810, 11)


In [6]:
param_dist = dict(
    max_depth=randint(low=8, high=18),
    min_samples_leaf=randint(2, 10),
)

random_search = RandomizedSearchCV(DecisionTreeRegressor(random_state=42), param_distributions=param_dist, verbose=2,
                          n_iter=50, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

random_search.fit(X_train_val, y_train_val)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   21.7s finished


RandomizedSearchCV(estimator=DecisionTreeRegressor(random_state=42), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff0a6a10100>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff0a6a104c0>},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [7]:
results_searchcv(random_search, X_test, y_test)

Best score:
-7.158411354504513e+17
Best parameters found:
{'max_depth': 17, 'min_samples_leaf': 9}
Score in test:
-6.753456996785446e+17
R^2 in test
0.8563522510656776
MAE in test:
432524248.89605796


In [8]:
# dot_data = tree.export_graphviz(random_search.best_estimator_, feature_names=X_train_val.columns, filled=True, out_file="tree.dot")

In [9]:
# graph = graphviz.Source(dot_data)
# graph